<a href="https://colab.research.google.com/github/mgetsova/kaggle/blob/main/IceCube_preds_on_trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

!pip install opendatasets
!pip install pandas

from pathlib import Path
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union
import numpy as np
import pandas as pd
import torch

from drive.MyDrive.IceCube.ice_transparency import ice_transparency
from drive.MyDrive.IceCube.prepare_sensors import prepare_sensors

INPUT_PATH = Path("drive/MyDrive/IceCube")
TRANSPARENCY_PATH = INPUT_PATH / "ice_transperancy.txt"
#FULL_TRAIN_META_PATH = INPUT_PATH / "train_meta.parquet"
TRAIN_PATH = INPUT_PATH / "train"
STARTER_PULSE_PATH = TRAIN_PATH / 'pulses'
STARTER_META_PATH = TRAIN_PATH / 'meta'

BATCH_FILE = 2

meta = pd.read_parquet(STARTER_META_PATH / f"batch_{BATCH_FILE}.parquet", columns=["event_id", 'azimuth', 'zenith'], engine="pyarrow", use_threads=True)
sensor_df = prepare_sensors(INPUT_PATH / "sensor_geometry.csv")
f_scattering, f_absorption = ice_transparency(TRANSPARENCY_PATH)
event_ids = meta.event_id
y = meta[['zenith', 'azimuth']].reset_index(drop=True)

import torch
#from torch.utils.data import Dataset
from torch_geometric.nn import knn_graph
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader

2.0.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
class IceCubeDataset(Dataset):
    def __init__(
        self,
        batch_id,
        event_ids,
        PATH_TO_BATCH_FILES,
        f_scattering,
        f_absorption,
        sensor_df,
        y,
        pulse_limit=300,
        transform=None,
        pre_transform=None,
        pre_filter=None,
    ):
        super().__init__(transform, pre_transform, pre_filter)
        self.event_ids = event_ids
        self.batch_df = pd.read_parquet(PATH_TO_BATCH_FILES / f"batch_{batch_id}.parquet")
        self.sensor_df = sensor_df
        self.pulse_limit = pulse_limit
        self.f_scattering = f_scattering
        self.f_absorption = f_absorption
        self.y = y
        #ice_transparency(TRANSPARENCY_PATH)

        self.batch_df["time"] = (self.batch_df["time"] - 1.0e04) / 3.0e4
        self.batch_df["charge"] = np.log10(self.batch_df["charge"]) / 3.0
        self.batch_df["auxiliary"] = self.batch_df["auxiliary"].astype(int) - 0.5

    def len(self):
        return len(self.event_ids)

    def get(self, idx):
        event_id = self.event_ids[idx]
        event = self.batch_df.loc[event_id]

        event = pd.merge(event, self.sensor_df, on="sensor_id")

        x = event[["x", "y", "z", "time", "charge", "qe", "auxiliary"]].values
        x = torch.tensor(x, dtype=torch.float32)
        data = Data(x=x, n_pulses=torch.tensor(x.shape[0], dtype=torch.int32))

        # Add ice transparency data
        z = data.x[:, 2].numpy()
        scattering = torch.tensor(self.f_scattering(z), dtype=torch.float32).view(-1, 1)
        # absorption = torch.tensor(self.f_absorption(z), dtype=torch.float32).view(-1, 1)

        data.x = torch.cat([data.x, scattering], dim=1)

        # Downsample the large events
        if data.n_pulses > self.pulse_limit:
            data.x = data.x[np.random.choice(data.n_pulses, self.pulse_limit)]
            data.n_pulses = torch.tensor(self.pulse_limit, dtype=torch.int32)

         # Builds graph from the k-nearest neighbours.
        data.edge_index = knn_graph(
            data.x[:, [0, 1, 2]],  # x, y, z
            k=8,
            batch=None,
            loop=False
        )

        if self.y is not None:
            y = self.y.loc[idx, :].values
            y = torch.tensor(y, dtype=torch.float32)
            data.y = y

        return data

2.0.0+cu118


In [65]:
dataset = IceCubeDataset(BATCH_FILE, list(event_ids), STARTER_PULSE_PATH, f_scattering, f_absorption, sensor_df, y)
dataset.len()
dataset.get(0).x.shape

torch.Size([74, 8])

In [66]:
from torch_geometric.utils.homophily import homophily

def calculate_xyzt_homophily(x, edge_index, batch):
    """Calculate xyzt-homophily from a batch of graphs.

    Homophily is a graph scalar quantity that measures the likeness of
    variables in nodes. Notice that this calculator assumes a special order of
    input features in x.

    Returns:
        Tuple, each element with shape [batch_size,1].
    """
    hx = homophily(edge_index, x[:, 0], batch).reshape(-1, 1)
    hy = homophily(edge_index, x[:, 1], batch).reshape(-1, 1)
    hz = homophily(edge_index, x[:, 2], batch).reshape(-1, 1)
    ht = homophily(edge_index, x[:, 3], batch).reshape(-1, 1)
    return hx, hy, hz, ht


from torch_geometric.nn import EdgeConv
from torch_geometric.nn.pool import knn_graph # differtent from one above????
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union
from torch import LongTensor, Tensor
from torch_geometric.typing import Adj

class DynEdgeConv(EdgeConv):
    """Dynamical edge convolution layer."""

    def __init__(
        self,
        nn: Callable,
        device,
        aggr: str = "max",
        nb_neighbors: int = 8,
        features_subset: Optional[Union[Sequence[int], slice]] = None,
        **kwargs: Any,
    ):
        """Construct `DynEdgeConv`.
        Args:
            nn: The MLP/torch.Module to be used within the `EdgeConv`.
            aggr: Aggregation method to be used with `EdgeConv`.
            nb_neighbors: Number of neighbours to be clustered after the
                `EdgeConv` operation.
            features_subset: Subset of features in `Data.x` that should be used
                when dynamically performing the new graph clustering after the
                `EdgeConv` operation. Defaults to all features.
            **kwargs: Additional features to be passed to `EdgeConv`.
        """
        # Check(s)
        if features_subset is None:
            features_subset = slice(None)  # Use all features
        assert isinstance(features_subset, (list, slice))

        # Base class constructor
        super().__init__(nn=nn, aggr=aggr, **kwargs)

        # Additional member variables
        self.device = device
        self.nb_neighbors = nb_neighbors
        self.features_subset = features_subset

    def forward(
        self, x: Tensor, edge_index: Adj, batch: Optional[Tensor] = None
    ) -> Tensor:

        """Forward pass."""
        # Standard EdgeConv forward pass
        x = super().forward(x, edge_index)
        # Recompute adjacency
        edge_index = knn_graph(x=x[:, self.features_subset], k=self.nb_neighbors, 
                               batch=batch).to(self.device)

        return x, edge_index

def define_mlp(nb_in, nb_hidden, nb_out, activation):
    return torch.nn.Sequential(torch.nn.Linear(nb_in, nb_hidden),
                               activation,
                               torch.nn.Linear(nb_hidden, nb_out),
                               activation)
    
import torch.nn as nn
from torch_scatter import scatter_max, scatter_mean, scatter_min, scatter_sum

GLOBAL_POOLINGS = {
    "min": scatter_min,
    "max": scatter_max,
    "sum": scatter_sum,
    "mean": scatter_mean,
}

class MyGNN(nn.Module):
    def __init__(self, nb_inputs, features_subset, device = 'cpu', nb_neighbors = 8, global_pooling_schemes = ['mean', 'min', 'max', 'sum'], 
                 add_global_variables_after_pooling = True, num_global_variables = 5):
        super().__init__()
        self.activation = torch.nn.LeakyReLU()
        self.device = device

        self.conv_layer1 = DynEdgeConv(define_mlp(nb_inputs*2, 128, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer2 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer3 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer4 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)

        self.post_processing = define_mlp(256*4 + nb_inputs, 336, 256, self.activation)

        nb_poolings = (len(global_pooling_schemes) if global_pooling_schemes else 1)
        nb_latent_features = 256 * nb_poolings
        if add_global_variables_after_pooling:
            nb_latent_features += num_global_variables

        #self.readout = torch.nn.Sequential(torch.nn.Linear(nb_latent_features, 128), self.activation)
        self.readout = define_mlp(nb_latent_features, 128, 2, self.activation)
        
        self._global_pooling_schemes = global_pooling_schemes
        self._add_global_variables_after_pooling = add_global_variables_after_pooling

    def _global_pooling(self, x: Tensor, batch: LongTensor) -> Tensor:
      """Perform global pooling."""
      assert self._global_pooling_schemes
      pooled = []
      for pooling_scheme in self._global_pooling_schemes:
        pooling_fn = GLOBAL_POOLINGS[pooling_scheme]
        pooled_x = pooling_fn(x, index=batch, dim=0)
        if isinstance(pooled_x, tuple) and len(pooled_x) == 2:
          pooled_x, _ = pooled_x
        pooled.append(pooled_x)

      return torch.cat(pooled, dim=1)

    def _calculate_global_variables(self, x: Tensor, edge_index: LongTensor, batch: LongTensor, 
                                    *additional_attributes: Tensor,) -> Tensor:
      """Calculate global variables."""
      # Calculate homophily (scalar variables)
      h_x, h_y, h_z, h_t = calculate_xyzt_homophily(x, edge_index, batch)
      # Calculate mean features
      global_means = scatter_mean(x, batch, dim=0)
      # Add global variables
      # global_variables = torch.cat([global_means, h_x, h_y, h_z, h_t] + [attr.unsqueeze(dim=1) for attr in additional_attributes], dim=1,)
      global_variables = torch.cat([h_x, h_y, h_z, h_t] + [attr.unsqueeze(dim=1) for attr in additional_attributes], dim=1,)


      return global_variables

    def forward(self, data: Data) -> Tensor:
      """Apply learnable forward pass."""
      # Convenience variables
      x, edge_index, batch = data.x, data.edge_index, data.batch
      global_variables = self._calculate_global_variables(x, edge_index, 
                                                          batch, torch.log10(data.n_pulses))

      # Distribute global variables out to each node
      if not self._add_global_variables_after_pooling:
        distribute = (batch.unsqueeze(dim=1) == torch.unique(batch).unsqueeze(dim=0)).type(torch.float) 
        global_variables_distributed = torch.sum(distribute.unsqueeze(dim=2) * 
                                                 global_variables.unsqueeze(dim=0), dim=1)
        x = torch.cat((x, global_variables_distributed), dim=1)

      # convolutions
      state_graphs = [x]
      conv_layers = [self.conv_layer1, self.conv_layer2, self.conv_layer3, self.conv_layer4]
      for layer in conv_layers:
        x, edge_index = layer(x, edge_index, batch)
        state_graphs.append(x)

      x = torch.cat(state_graphs, dim=1)
      # post processing
      x = self.post_processing(x)

      # global pooling
      if self._global_pooling_schemes:
        x = self._global_pooling(x, batch=batch)
        if self._add_global_variables_after_pooling:
          x = torch.cat([x, global_variables], dim=1)

      # readout layer
      x = self.readout(x)
      return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
compute_knn_with = [0, 1, 2, 3] # x, y, z, time
model = MyGNN(8, compute_knn_with, device='cuda')
print('using ', device)

using  cpu


In [67]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))
 

In [68]:
check_loader = DataLoader(dataset, batch_size=1, num_workers=1)

In [77]:
#dataset = IceCubeDataset(BATCH_FILE, event_ids, STARTER_PULSE_PATH, f_scattering, f_absorption, sensor_df, y)
MODEL_PATH = INPUT_PATH / "first_model_epoch0.pt"
model = torch.load(MODEL_PATH, map_location=torch.device('cpu'))

def infer(model, loader, device="cpu"):
    model.to(device)
    model.eval()

    with torch.no_grad():
      #preds = model(loader)
      #for batch in tqdm(loader, desc="test"): 
      #for batch_idx, data in enumerate(loader):
      for batch in loader:
        batch = batch.to(device)
        preds = model(batch)

    return preds

infer(model, check_loader, device='cpu')


RuntimeError: ignored